In [1]:
import pandas as pd
import numpy as np
from os import listdir
from data_loaders import zillow_pipeline, acs_pipeline
import collections
import json
import geopandas as gpd

In [2]:
long_dfs = []

# Load Zillow

In [3]:
path = '../data/Zillow/Zip_MedianRentalPrice_AllHomes.csv'
year_moved_start = 2010
year_moved_end = 2019

# load data
df = pd.read_csv(path, encoding = 'mac_roman', dtype={'RegionName':str})
years = range(year_moved_start, year_moved_end + 1)
df['zipcode'] = df['RegionName']

dfs = []
for year in years:
    for month in range(1,13):
        month_str = '0' + str(month) if month < 10 else str(month)
        year_month_clm = f"{year}-{month_str}"
        try:
            long_df_year_month = (
                df[[year_month_clm, 'zipcode']]
                .copy()
                .rename(index=str, columns={
                    year_month_clm: 'median_rent'
                })
            )
            long_df_year_month['time'] = year+month/12
            long_df_year_month['source'] = 'zillow_zipcode'

            dfs.append(long_df_year_month)
        except KeyError:
            pass
zillow_df = pd.concat(dfs)
zillow_df.sample(10)

,median_rent,zipcode,time,source
783,1150.0,70506,2018.583333,zillow_zipcode
3076,NaN,23504,2015.750000,zillow_zipcode
1077,1985.0,77024,2018.416667,zillow_zipcode
348,NaN,19120,2012.166667,zillow_zipcode
3369,NaN,93109,2017.500000,zillow_zipcode
1067,1400.0,84790,2017.666667,zillow_zipcode
684,945.0,75040,2011.166667,zillow_zipcode
2363,3300.0,91007,2018.833333,zillow_zipcode
1985,3600.0,20007,2018.333333,zillow_zipcode
2309,NaN,91356,2013.833333,zillow_zipcode


# Load ACS 5 year recent mover

In [4]:
path_to_dir = '../data/ACS-5year-zipcode/'

filenames = {
    'nhgis0010_ds192_20125_2012_zcta.csv':'RGL',
    'nhgis0010_ds234_20175_2017_zcta.csv':'AIND',
    'nhgis0010_ds207_20145_2014_zcta.csv':'ABZA',
    'nhgis0010_ds216_20155_2015_zcta.csv':'AD85',
    'nhgis0010_ds226_20165_2016_zcta.csv':'AGQV',
    'nhgis0010_ds202_20135_2013_zcta.csv':'U21',
    'nhgis0010_ds185_20115_2011_zcta.csv':'NAM'
}


In [5]:
zipcode_clm = 'ZCTA5A'
time_end = range(2011, 2018)
dfs=[]
for idx, f in enumerate(filenames):
    path = path_to_dir + f
    df = pd.read_csv(path, encoding = 'mac_roman', dtype={'ZCTA5A':str})
    df_clean = pd.DataFrame({
        'zipcode':df[zipcode_clm],
        'time':int(f.split('_')[3]),
        'source':'acs_5year_zipcode_recent_mover',
        'median_rent':df[filenames[f]+'E002'],
        'median_rent_moe':df[filenames[f]+'M002']
    })
    dfs.append(df_clean)
acs_5y_recent_df = pd.concat(dfs)
acs_5y_recent_df = acs_5y_recent_df[acs_5y_recent_df.median_rent != 2001]

acs_5y_recent_df[acs_5y_recent_df.zipcode == '20004']

,zipcode,time,source,median_rent,median_rent_moe
6219,20004,2017,acs_5year_zipcode_recent_mover,2534.0,584.0
6219,20004,2015,acs_5year_zipcode_recent_mover,2844.0,1264.0
6219,20004,2016,acs_5year_zipcode_recent_mover,2977.0,472.0


# Load ACS 1 year recent mover

In [142]:
path_to_crosswalk = '../data/ACS-1year-county/crosswalk/ZIP_COUNTY_062019.csv'
zip_tract_crosswalk = pd.read_csv(path_to_crosswalk, dtype = {'county':str,'zip':str})

In [147]:
zip_tract_crosswalk[zip_tract_crosswalk.zip == '98068']

,zip,county,res_ratio,bus_ratio,oth_ratio,tot_ratio
5943,98068,53037,0.0,0.0,0.5,0.5
53700,98068,53033,0.0,0.0,0.5,0.5


In [148]:
?pd.DataFrame.drop_duplicates

In [150]:
idx = zip_tract_crosswalk.groupby(['zip'])['tot_ratio'].transform(max) == zip_tract_crosswalk['tot_ratio']
zip_tract_crosswalk = zip_tract_crosswalk[idx].drop_duplicates(subset = 'zip')

In [152]:
zip_tract_crosswalk = zip_tract_crosswalk[['county','zip']]
zip_tract_crosswalk.columns = ['fips','zipcode']
zip_tract_crosswalk.head()

,fips,zipcode
0,72013,00616
2,72111,00624
3,72013,00652
4,72067,00660
5,72083,00670


In [153]:
path_to_dir = '../data/ACS-1year-county/'

filenames = {
    'nhgis0010_ds232_2017_county.csv':'AHRA',
    'nhgis0010_ds205_2014_county.csv':'AA40',
    'nhgis0010_ds199_2013_county.csv':'S65',
    'nhgis0010_ds183_2011_county.csv':'MH9',
    'nhgis0010_ds189_2012_county.csv':'PH8',
    'nhgis0010_ds214_2015_county.csv':'ADEU',
    'nhgis0010_ds223_2016_county.csv':'AFV4'
}

In [156]:
state_code_clm = 'STATEA'
county_code_clm = 'COUNTYA'
time = range(2011, 2018)
dfs=[]
for idx, f in enumerate(filenames):
    path = path_to_dir + f
    df = pd.read_csv(path, encoding = 'mac_roman', dtype={state_code_clm:str,county_code_clm:str})
    df_clean = pd.DataFrame({
        'fips':df[state_code_clm]+df[county_code_clm],
        'time': int(f.split('_')[2]),
        'source':'acs_1year_county_recent_mover',
        'median_rent':df[filenames[f]+'E002'],
        'median_rent_moe':df[filenames[f]+'M002']
    })
    df_clean = (
        df_clean
        .merge(zip_tract_crosswalk)
        .drop('fips',axis = 1)
    )
    dfs.append(df_clean)
acs_1y_recent_df = pd.concat(dfs)

In [157]:
acs_1y_recent_df = acs_1y_recent_df[acs_1y_recent_df.median_rent != 2001]
acs_1y_recent_df[acs_1y_recent_df.zipcode == '20004']

,time,source,median_rent,median_rent_moe,zipcode
4299,2017,acs_1year_county_recent_mover,1823.0,42.0,20004
4296,2014,acs_1year_county_recent_mover,1592.0,53.0,20004
4296,2013,acs_1year_county_recent_mover,1554.0,54.0,20004
4296,2011,acs_1year_county_recent_mover,1350.0,36.0,20004
4296,2012,acs_1year_county_recent_mover,1478.0,43.0,20004
4299,2015,acs_1year_county_recent_mover,1865.0,82.0,20004
4299,2016,acs_1year_county_recent_mover,1723.0,68.0,20004


# Load HUD Fair Market Rents

In [158]:
SAFMR_df = (
    pd.read_csv('../data/FMR/SAFMR/SAFMR_Consolidated.csv',dtype = {'zipcode':str,'median_rent':int})
    .groupby(['time','zipcode'],as_index=False)
    .agg({'median_rent':np.mean,'source': lambda x: "SAFMR"})
)
SAFMR_df

,time,zipcode,median_rent,source
0,2012,00602,470.0,SAFMR
1,2012,00603,470.0,SAFMR
2,2012,00604,470.0,SAFMR
3,2012,00605,470.0,SAFMR
4,2012,00606,430.0,SAFMR
5,2012,00610,430.0,SAFMR
6,2012,00612,480.0,SAFMR
7,2012,00613,480.0,SAFMR
8,2012,00616,440.0,SAFMR
9,2012,00617,470.0,SAFMR


# Combine Data and Save

In [159]:
df_all = (
    pd.concat([
        zillow_df,
        acs_1y_recent_df,
        acs_5y_recent_df,
        SAFMR_df
    ], sort = False)
    .dropna(subset=['median_rent'])
    .drop_duplicates()
    .sort_values('time')
)
df_all.head()

,median_rent,zipcode,time,source,median_rent_moe
82,1195.0,23462,2010.166667,zillow_zipcode,NaN
494,3200.0,85255,2010.166667,zillow_zipcode,NaN
571,1900.0,33178,2010.166667,zillow_zipcode,NaN
899,2000.0,33180,2010.166667,zillow_zipcode,NaN
1505,795.0,32547,2010.166667,zillow_zipcode,NaN


check for duplicates

In [160]:
df_dup_count = df_all.groupby(['zipcode','source','time']).count().reset_index()
df_dup = df_dup_count[df_dup_count.median_rent > 1]
df_dup

,zipcode,source,time,median_rent,median_rent_moe


In [161]:
df_all.to_csv('../data/clean/clean_rents.csv', index=False)

In [162]:
records = (df_all
          # [df_all.zipcode.apply(lambda x: str(x)[:2] == '20')]
           .fillna('null')
           .groupby(['zipcode','source'], as_index=False)
           .apply(lambda x: x[['median_rent','time','median_rent_moe']]
                  .to_dict('r'))
 #.to_json('../data/clean/clean_rents.json', orient='index')
)


In [163]:
sources = df_all.source.unique()
mapping = dict(zip(sources, ['zillow','acs5','acs1','safmr']))
mapping

{'zillow_zipcode': 'zillow',
 'acs_5year_zipcode_recent_mover': 'acs5',
 'acs_1year_county_recent_mover': 'acs1',
 'SAFMR': 'safmr'}

In [164]:
nested_dict = collections.defaultdict(dict)

for keys, value in records.iteritems():
    nested_dict[keys[0]][mapping[keys[1]]] = value

for zipcode in nested_dict:
    for source in mapping.values():
        if source not in nested_dict[zipcode]:
            nested_dict[zipcode][source] = []

In [165]:
nested_dict['22204']['acs1']

[{'median_rent': 1724.0, 'time': 2011.0, 'median_rent_moe': 53.0},
 {'median_rent': 1832.0, 'time': 2012.0, 'median_rent_moe': 58.0},
 {'median_rent': 1942.0, 'time': 2013.0, 'median_rent_moe': 65.0},
 {'median_rent': 1960.0, 'time': 2014.0, 'median_rent_moe': 62.0},
 {'median_rent': 1962.0, 'time': 2015.0, 'median_rent_moe': 142.0},
 {'median_rent': 1961.0, 'time': 2016.0, 'median_rent_moe': 58.0},
 {'median_rent': 1992.0, 'time': 2017.0, 'median_rent_moe': 67.0}]

In [166]:
nested_dict['20002']['acs5']

[{'median_rent': 1192.0, 'time': 2011.0, 'median_rent_moe': 48.0},
 {'median_rent': 1413.0, 'time': 2012.0, 'median_rent_moe': 131.0},
 {'median_rent': 1437.0, 'time': 2013.0, 'median_rent_moe': 98.0},
 {'median_rent': 1476.0, 'time': 2014.0, 'median_rent_moe': 80.0},
 {'median_rent': 2050.0, 'time': 2015.0, 'median_rent_moe': 241.0},
 {'median_rent': 1939.0, 'time': 2016.0, 'median_rent_moe': 143.0},
 {'median_rent': 1956.0, 'time': 2017.0, 'median_rent_moe': 86.0}]

In [168]:
with open('../data/clean/clean_rents.json', 'w') as outfile:  
    json.dump(nested_dict, outfile)

In [169]:
with open('../data/clean/clean_rents_20001.json', 'w') as outfile:  
    json.dump(nested_dict['20001'], outfile)

# make zipcode geoJSON

In [109]:
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon


In [110]:
zip_shp = gpd.read_file('../data/zipcodes/cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')

In [111]:
dc_zip = (
    zip_shp
    [zip_shp.ZCTA5CE10.apply(lambda x: x[:2] == "20")]
    .merge(rent_change, left_on = 'ZCTA5CE10', right_on = 'zipcode')
    [list(rent_change.columns) + ['geometry']]
)

In [112]:
dc_zip["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon \
    else feature for feature in dc_zip["geometry"]]

In [113]:
dc_zip.to_file("../data/zipcodes/zipcodes.geojson", driver='GeoJSON')